# Initial Imports

In [1]:
import random
random.seed(42)
import numpy as np
from games4e import Game

# Define Mancala Game and Mancala State

In [2]:
class MancalaState:
    def __init__(self, board = [], to_move = 1):
        """
        The constructor for the MancalaBoard class defines several instance variables:

        board: This data structure is responsible for managing the Mancala board.
        to_move: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        """

        self.board = board
        self.to_move = to_move

    def new(self):
        return MancalaState(self.board.copy(), self.to_move)

    def switch_player(self):
        """
        Switches `to_move` between 1 and 2

        Parameters:
            None

        Returns:
            None
        """
    
        offset = 1

        self.to_move = self.to_move % 2 + offset

class Mancala(Game):
    def __init__(self, pits_per_player = 6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        initial_board = [stones_per_pit] * ((pits_per_player + 1) * 2)
        # self.moves = []
        self.p1_pits_index = [0, self.pits_per_player - 1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player + 1, len(initial_board) - 2]
        self.p2_mancala_index = len(initial_board) - 1
        self.p1_turn_counter = 0
        self.p2_turn_counter = 0

        # Initialize Mancala to 0 for both players
        initial_board[self.p1_mancala_index] = 0
        initial_board[self.p2_mancala_index] = 0

        self.initial = MancalaState(initial_board)

    def actions(self, state):
        """
        Return list of possible pits that the current player can play.
        """

        return self.get_valid_moves(state)

    def result(self, state, pit):
        """
        Return the new board state after playing `pit` on `board`.

        Parameters:
            board (list) - The current state of the board
            pit (int) - The pit to play on the display board interface

        Returns:
            list - The new board state after playing `pit` on `board`
        """

        player = self.to_move(state)
        pit_index = self.get_pit_index(player, pit)

        state_copy = state.new()

        self.distribute_stones(state_copy, pit_index)
        state_copy.switch_player()
        
        return state_copy
    
    def utility(self, state, player):
        """
        Returns the utility of the game board for the current player (number of stones in Mancala).
        """

        player_mancala_index = self.get_player_mancala_index(player)

        return state.board[player_mancala_index]
    
    def terminal_test(self, state):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        
        no_stones_in_player_1_pits = all([state.board[pit] == 0 for pit in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)])
        no_stones_in_player_2_pits = all([state.board[pit] == 0 for pit in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)])

        return no_stones_in_player_1_pits or no_stones_in_player_2_pits
    
    def display(self, state):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = state.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = state.board[self.p1_mancala_index]
        player_2_pits = state.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = state.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.to_move(state) == 1 else 'P2'
        print('Turn: ' + turn)
    
    def play_game(self, depth, *players):
        """
        Play an n-person, move-alternating game.
        """

        state = self.initial
        while True:
            for (i, player) in enumerate(players):
                move = player(state, self, depth)
                state = self.result(state, move)
                if i + 1 == 1:
                    self.p1_turn_counter += 1
                else:
                    self.p2_turn_counter += 1

                # self.display(state)

                if self.terminal_test(state):
                    self.clean_stones(state)
                    # self.display(state)
                    return self.utility(state, 1), self.utility(state, 2)



    #
    # Helper Functions
    #

    def valid_move(self, state, pit_index):
        """
        Function to check if the pit chosen by `to_move` is a valid move.
        """

        pit_has_stones = state.board[pit_index] > 0
        
        player_pits = self.get_player_pits(self.to_move(state))
        indexOfFirstPit = player_pits[0]
        indexOfLastPit = player_pits[1]
        pit_on_current_player_side = indexOfFirstPit <= pit_index <= indexOfLastPit

        pit_is_not_mancala = (pit_index != self.p1_mancala_index) and (pit_index != self.p2_mancala_index)

        return pit_has_stones and pit_on_current_player_side and pit_is_not_mancala
    
    def get_valid_moves(self, state):
        """
        Returns a list of pit values (between 1 and `pits_per_player` inclusive) that can be played on `to_move`'s turn

        Parameters:
            None

        Returns:
            List of int - Pit values (between 1 and `pits_per_player` inclusive) that can be played on `to_move`'s turn
        """

        current_player = self.to_move(state)
        player_pits = self.get_player_pits(current_player)
        return [self.get_pit_from_index(current_player, pit_index) for pit_index in range(player_pits[0], player_pits[1] + 1) if self.valid_move(state, pit_index)]

    def get_player_pits(self, player):
        """
        Return the repective pit indices list for `to_move` with respect to the `board` list.

        Parameters:
            None

        Returns:
            List of int - List of pit indices for `to_move` with respect to the `board` list
        """
        
        return self.p1_pits_index if player == 1 else self.p2_pits_index

    def get_pit_index(self, player, pit):
        """
        Returns the index of `pit` with respect to the `board` list

        Parameters:
            pit (int) - A value between 1 and `pits_per_player` inclusive

        Returns:
            int - A value between 0 and len(board) inclusive which represents an index in `board`
        """
        
        startOfPlayer2Pits = self.p2_pits_index[0]

        match (player):
            case 1:
                return pit - 1
            case 2:
                return pit + startOfPlayer2Pits - 1
        
    def in_opponent_mancala(self, player, pit_index):
        """
        Check if the current `pit_index` represents the opponent's mancala pit.

        Parameters:
            pit_index (int) - A value between 0 and len(board) inclusive which represents the index of a pit on `to_move`'s side

        Returns:
            Boolean - True if `pit_index` is the opponent's mancala pit and False otherwise.
        """
        
        return (player == 1 and pit_index == self.p2_mancala_index) or (player == 2 and pit_index == self.p1_mancala_index)

    def distribute_stones(self, state, pit_index):
        """
        Takes stones from `pit_index` on board and distributes them throughout the board in accordance to the Mancala rules.

        Parameters:
            pit_index (int) - A value between 0 and len(board) inclusive which represents the index of a pit on `to_move`'s side

        Returns:
            None
        """
        
        player = self.to_move(state)
        current_stones = state.board[pit_index]
        state.board[pit_index] = 0

        while current_stones > 0:
            pit_index = (pit_index + 1) % len(state.board)

            if self.in_opponent_mancala(player, pit_index):
                continue

            state.board[pit_index] += 1
            current_stones -= 1

        if self.valid_move(state, pit_index) and state.board[pit_index] == 1:
            # Last stone placed in `pit_index` on `to_move`'s side
            opposite_pit_index = self.get_opposite_pit_index(state, pit_index)
            current_player_mancala_index = self.get_player_mancala_index(player)

            state.board[current_player_mancala_index] += state.board[pit_index] + state.board[opposite_pit_index]
            state.board[pit_index] = 0
            state.board[opposite_pit_index] = 0

    def get_pit_from_index(self, player, pit_index):
        """
        Returns the pit value of `pit_index` with respect to the `board` list and `to_move`

        Parameters:
            pit_index (int) - A value between 0 and len(board) inclusive which represents an index in `board`

        Returns:
            int - A value betwen 1 and `pits_per_player` inclusive
        """

        startOfPlayer2Pits = self.p2_pits_index[0]
        
        match(player):
            case 1:
                return pit_index + 1
            case 2:
                return pit_index - startOfPlayer2Pits + 1
            
    def get_opposite_pit_index(self, state, pit_index):
        """
        Return the pit index of the pit opposite to `pit_index`

        Parameters:
            pit_index (int) - A value between 0 and len(board) inclusive which represents an index in `board`

        Returns:
            int - A value between 0 and len(board) inclusive which represents an index in `board` representing the opposite pit
        """

        pit = self.get_pit_from_index(self.to_move(state), pit_index)
        state.switch_player()
        opposite_pit = self.pits_per_player - pit + 1
        opposite_pit_index = self.get_pit_index(self.to_move(state), opposite_pit)
        state.switch_player()

        return opposite_pit_index
    
    def get_player_mancala_index(self, player):
        """
        Returns the index in `board` that corresponds to `to_move`'s mancala pit

        Parameters:
            None

        Returns:
            int - Index of `to_move`'s mancala in the board
        """

        match(player):
            case 1:
                return self.p1_mancala_index
            case 2:
                return self.p2_mancala_index
            
    def clean_stones(self, state):
        """
        Move remaining stones to the respective player's Mancalla following the end of the game.

        Parameters:
            None

        Returns:
            None
        """

        if all([state.board[pit] == 0 for pit in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)]):
            # Player 1 has no stones left
            for pit_index in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                state.board[self.p2_mancala_index] += state.board[pit_index]
                state.board[pit_index] = 0
        else:
            # Player 2 has no stones left
            for pit_index in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                state.board[self.p1_mancala_index] += state.board[pit_index]
                state.board[pit_index] = 0

# Defining Methods for Different Types of Players

In [3]:
def random_player(state, game, depth):
    return random.choice(game.actions(state))

def player(search_algorithm):
    return lambda state, game, depth: search_algorithm(state, game, depth)

# Define Minmax and Alpha Beta Search Methods From AIMA Python Library

In [4]:
def minmax_decision(state, game, depth):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state, depth):
        if depth == 0 or game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), depth - 1))
        return v

    def min_value(state, depth):
        if depth == 0 or game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), depth - 1))
        return v

    # Body of minmax_decision:
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a), depth))

def alpha_beta_search(state, game, depth):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if depth == 0 or game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth - 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if depth == 0 or game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth - 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, depth)
        if v > best_score:
            best_score = v
            best_action = a
            
    return best_action

# Simulations

## Define Method to Simulate Games

In [5]:
def simulate(num_games, player_1, player_2, depth):
    wins = 0
    losses = 0
    ties = 0
    p1_turns_by_game = []
    p2_turns_by_game = []

    for _ in range(num_games):
        game = Mancala()
        (p1_score, p2_score) = game.play_game(depth, player_1, player_2)

        p1_turns_by_game.append(game.p1_turn_counter)
        p2_turns_by_game.append(game.p2_turn_counter)

        if p1_score > p2_score:
            wins += 1
        elif p2_score > p1_score:
            losses += 1
        else:
            ties += 1

    print(f"Games Won: {(wins / num_games):.1%}")
    print(f"Games Lost: {(losses / num_games):.1%}")
    print(f"Games Tied: {(ties / num_games):.1%}")
    print("Average Player 1 Turns per Game:", sum(p1_turns_by_game) / len(p1_turns_by_game))
    print("Average Player 2 Turns per Game:", sum(p2_turns_by_game) / len(p2_turns_by_game))

def simulate_variable_depth(num_games, player_1, player_2, depth_max):
    wins = 0
    losses = 0
    ties = 0
    p1_turns_by_game = []
    p2_turns_by_game = []

    for _ in range(num_games):
        game = Mancala()
        (p1_score, p2_score) = game.play_game(random.randrange(depth_max) + 1, player_1, player_2)

        p1_turns_by_game.append(game.p1_turn_counter)
        p2_turns_by_game.append(game.p2_turn_counter)

        if p1_score > p2_score:
            wins += 1
        elif p2_score > p1_score:
            losses += 1
        else:
            ties += 1

    print(f"Games Won: {(wins / num_games):.1%}")
    print(f"Games Lost: {(losses / num_games):.1%}")
    print(f"Games Tied: {(ties / num_games):.1%}")
    print("Average Player 1 Turns per Game:", sum(p1_turns_by_game) / len(p1_turns_by_game))
    print("Average Player 2 Turns per Game:", sum(p2_turns_by_game) / len(p2_turns_by_game))

## 100 Games Random Player vs Random Player

In [6]:
simulate(100, random_player, random_player, 0)

Games Won: 49.0%
Games Lost: 48.0%
Games Tied: 3.0%
Average Player 1 Turns per Game: 21.06
Average Player 2 Turns per Game: 20.52


## 100 Games Minimax AI vs Random with Plies Ranging Between 1 and 10

In [ ]:
simulate_variable_depth(100, player(minmax_decision), random_player, 7)

## 100 Games Minimax AI vs Random at Depth of 5 Plies

In [ ]:
simulate(1, player(minmax_decision), random_player, 7)

Games Won: 100.0%
Games Lost: 0.0%
Games Tied: 0.0%
Average Player 1 Turns per Game: 13.0
Average Player 2 Turns per Game: 12.0


In [ ]:
simulate(100, player(minmax_decision), random_player, 5)

Games Won: 89.0%
Games Lost: 6.0%
Games Tied: 5.0%
Average Player 1 Turns per Game: 15.04
Average Player 2 Turns per Game: 14.46


# 100 Games Alpha-Beta vs Random at Depth of 5 Plies

In [ ]:
simulate(100, player(alpha_beta_search), random_player, 5)

Games Won: 94.0%
Games Lost: 2.0%
Games Tied: 4.0%
Average Player 1 Turns per Game: 15.25
Average Player 2 Turns per Game: 14.65


# Extra Credit

# Modify Utility Function to Be Difference in Stones in Mancala

In [ ]:
class MancalaState:
    def __init__(self, board = [], to_move = 1):
        """
        The constructor for the MancalaBoard class defines several instance variables:

        board: This data structure is responsible for managing the Mancala board.
        to_move: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        """

        self.board = board
        self.to_move = to_move

    def new(self):
        return MancalaState(self.board.copy(), self.to_move)

    def switch_player(self):
        """
        Switches `to_move` between 1 and 2

        Parameters:
            None

        Returns:
            None
        """
    
        offset = 1

        self.to_move = self.to_move % 2 + offset

class Mancala(Game):
    def __init__(self, pits_per_player = 6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        initial_board = [stones_per_pit] * ((pits_per_player + 1) * 2)
        # self.moves = []
        self.p1_pits_index = [0, self.pits_per_player - 1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player + 1, len(initial_board) - 2]
        self.p2_mancala_index = len(initial_board) - 1
        self.p1_turn_counter = 0
        self.p2_turn_counter = 0

        # Initialize Mancala to 0 for both players
        initial_board[self.p1_mancala_index] = 0
        initial_board[self.p2_mancala_index] = 0

        self.initial = MancalaState(initial_board)

    def actions(self, state):
        """
        Return list of possible pits that the current player can play.
        """

        return self.get_valid_moves(state)

    def result(self, state, pit):
        """
        Return the new board state after playing `pit` on `board`.

        Parameters:
            board (list) - The current state of the board
            pit (int) - The pit to play on the display board interface

        Returns:
            list - The new board state after playing `pit` on `board`
        """

        player = self.to_move(state)
        pit_index = self.get_pit_index(player, pit)

        state_copy = state.new()

        self.distribute_stones(state_copy, pit_index)
        state_copy.switch_player()
        
        return state_copy
    
    def utility(self, state, player):
        """
        Returns the utility of the game board for the current player (number of stones in Mancala).
        """

        player_mancala_index = self.get_player_mancala_index(player)
        opponent = 2 if player == 1 else 1
        opponent_mancala_index = self.get_player_mancala_index(opponent)

        return state.board[player_mancala_index] - state.board[opponent_mancala_index]
    
    def terminal_test(self, state):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        
        no_stones_in_player_1_pits = all([state.board[pit] == 0 for pit in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)])
        no_stones_in_player_2_pits = all([state.board[pit] == 0 for pit in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)])

        return no_stones_in_player_1_pits or no_stones_in_player_2_pits
    
    def display(self, state):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = state.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = state.board[self.p1_mancala_index]
        player_2_pits = state.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = state.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.to_move(state) == 1 else 'P2'
        print('Turn: ' + turn)
    
    def play_game(self, depth, *players):
        """
        Play an n-person, move-alternating game.
        """

        state = self.initial
        while True:
            for (i, player) in enumerate(players):
                move = player(state, self, depth)
                state = self.result(state, move)
                if i + 1 == 1:
                    self.p1_turn_counter += 1
                else:
                    self.p2_turn_counter += 1

                # self.display(state)

                if self.terminal_test(state):
                    self.clean_stones(state)
                    # self.display(state)
                    return self.utility(state, 1), self.utility(state, 2)



    #
    # Helper Functions
    #

    def valid_move(self, state, pit_index):
        """
        Function to check if the pit chosen by `to_move` is a valid move.
        """

        pit_has_stones = state.board[pit_index] > 0
        
        player_pits = self.get_player_pits(self.to_move(state))
        indexOfFirstPit = player_pits[0]
        indexOfLastPit = player_pits[1]
        pit_on_current_player_side = indexOfFirstPit <= pit_index <= indexOfLastPit

        pit_is_not_mancala = (pit_index != self.p1_mancala_index) and (pit_index != self.p2_mancala_index)

        return pit_has_stones and pit_on_current_player_side and pit_is_not_mancala
    
    def get_valid_moves(self, state):
        """
        Returns a list of pit values (between 1 and `pits_per_player` inclusive) that can be played on `to_move`'s turn

        Parameters:
            None

        Returns:
            List of int - Pit values (between 1 and `pits_per_player` inclusive) that can be played on `to_move`'s turn
        """

        current_player = self.to_move(state)
        player_pits = self.get_player_pits(current_player)
        return [self.get_pit_from_index(current_player, pit_index) for pit_index in range(player_pits[0], player_pits[1] + 1) if self.valid_move(state, pit_index)]

    def get_player_pits(self, player):
        """
        Return the repective pit indices list for `to_move` with respect to the `board` list.

        Parameters:
            None

        Returns:
            List of int - List of pit indices for `to_move` with respect to the `board` list
        """
        
        return self.p1_pits_index if player == 1 else self.p2_pits_index

    def get_pit_index(self, player, pit):
        """
        Returns the index of `pit` with respect to the `board` list

        Parameters:
            pit (int) - A value between 1 and `pits_per_player` inclusive

        Returns:
            int - A value between 0 and len(board) inclusive which represents an index in `board`
        """
        
        startOfPlayer2Pits = self.p2_pits_index[0]

        match (player):
            case 1:
                return pit - 1
            case 2:
                return pit + startOfPlayer2Pits - 1
        
    def in_opponent_mancala(self, player, pit_index):
        """
        Check if the current `pit_index` represents the opponent's mancala pit.

        Parameters:
            pit_index (int) - A value between 0 and len(board) inclusive which represents the index of a pit on `to_move`'s side

        Returns:
            Boolean - True if `pit_index` is the opponent's mancala pit and False otherwise.
        """
        
        return (player == 1 and pit_index == self.p2_mancala_index) or (player == 2 and pit_index == self.p1_mancala_index)

    def distribute_stones(self, state, pit_index):
        """
        Takes stones from `pit_index` on board and distributes them throughout the board in accordance to the Mancala rules.

        Parameters:
            pit_index (int) - A value between 0 and len(board) inclusive which represents the index of a pit on `to_move`'s side

        Returns:
            None
        """
        
        player = self.to_move(state)
        current_stones = state.board[pit_index]
        state.board[pit_index] = 0

        while current_stones > 0:
            pit_index = (pit_index + 1) % len(state.board)

            if self.in_opponent_mancala(player, pit_index):
                continue

            state.board[pit_index] += 1
            current_stones -= 1

        if self.valid_move(state, pit_index) and state.board[pit_index] == 1:
            # Last stone placed in `pit_index` on `to_move`'s side
            opposite_pit_index = self.get_opposite_pit_index(state, pit_index)
            current_player_mancala_index = self.get_player_mancala_index(player)

            state.board[current_player_mancala_index] += state.board[pit_index] + state.board[opposite_pit_index]
            state.board[pit_index] = 0
            state.board[opposite_pit_index] = 0

    def get_pit_from_index(self, player, pit_index):
        """
        Returns the pit value of `pit_index` with respect to the `board` list and `to_move`

        Parameters:
            pit_index (int) - A value between 0 and len(board) inclusive which represents an index in `board`

        Returns:
            int - A value betwen 1 and `pits_per_player` inclusive
        """

        startOfPlayer2Pits = self.p2_pits_index[0]
        
        match(player):
            case 1:
                return pit_index + 1
            case 2:
                return pit_index - startOfPlayer2Pits + 1
            
    def get_opposite_pit_index(self, state, pit_index):
        """
        Return the pit index of the pit opposite to `pit_index`

        Parameters:
            pit_index (int) - A value between 0 and len(board) inclusive which represents an index in `board`

        Returns:
            int - A value between 0 and len(board) inclusive which represents an index in `board` representing the opposite pit
        """

        pit = self.get_pit_from_index(self.to_move(state), pit_index)
        state.switch_player()
        opposite_pit = self.pits_per_player - pit + 1
        opposite_pit_index = self.get_pit_index(self.to_move(state), opposite_pit)
        state.switch_player()

        return opposite_pit_index
    
    def get_player_mancala_index(self, player):
        """
        Returns the index in `board` that corresponds to `to_move`'s mancala pit

        Parameters:
            None

        Returns:
            int - Index of `to_move`'s mancala in the board
        """

        match(player):
            case 1:
                return self.p1_mancala_index
            case 2:
                return self.p2_mancala_index
            
    def clean_stones(self, state):
        """
        Move remaining stones to the respective player's Mancalla following the end of the game.

        Parameters:
            None

        Returns:
            None
        """

        if all([state.board[pit] == 0 for pit in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)]):
            # Player 1 has no stones left
            for pit_index in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                state.board[self.p2_mancala_index] += state.board[pit_index]
                state.board[pit_index] = 0
        else:
            # Player 2 has no stones left
            for pit_index in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                state.board[self.p1_mancala_index] += state.board[pit_index]
                state.board[pit_index] = 0